In [ ]:
import re
import telebot
import wikipedia
from bs4 import BeautifulSoup

API_TOKEN = '6050824606:AAHNac-LYZ5rKWrcPauk1W7CsqqRzNCV0HE'


class WikiBot:
    def __init__(self):
        self.wiki = wikipedia  


def wikiparse(page):
    wikitext = page.content[:1000]  # ограничение длины сообщения
    wikimas = wikitext.split('.')
    wikimas = wikimas[:-1]
    wikitext2 = ''
    for x in wikimas:
        if not ('==' in x):
            if len((x.strip())) > 3:
                wikitext2 = wikitext2 + x + '.'
        else:
            break
    # удаление разметки
    wikitext2 = re.sub('\([^()]*\)', '', wikitext2)
    wikitext2 = re.sub('\([^()]*\)', '', wikitext2)
    wikitext2 = re.sub('\{[^\{\}]*\}', '', wikitext2)
    # возвращаем текстовую строку
    return wikitext2


def getwiki(wiki, text):
    try:
        direct_search = wiki.page(text, auto_suggest=False)
        wik = wikiparse(direct_search)
        return wik
    # Обработка исключения, которое может вернуть модуль Википедии
    except wikipedia.exceptions.DisambiguationError as e:
        opt = e.options
        wik = "Извини, твой запрос слишком двусмысленный!\n" \
              "'{0}' может относиться к:\n" \
              "\n<b>{1}</b>\n" \
              "<b>{2}</b>\n" \
              "<b>{3}</b>\n" \
              "<b>{4}</b>\n" \
              "<b>{5}</b>\n" \
              "\nПопробуй поиск по одному из приведенных выше предложений.".format(e.title,
                                                                     opt[0],
                                                                     opt[1],
                                                                     opt[2],
                                                                     opt[3],
                                                                     opt[4])
        return wik
    except wikipedia.exceptions.PageError:
        try:
            suggest_search = wiki.page(text, auto_suggest=True)
            wik = wikiparse(suggest_search)
            return wik
        except wikipedia.exceptions.PageError:
            return 'Извините, я ничего не могу найти по этому вопросу😔. Можете поискать в интернете по ссылке:' + 'https://yandex.ru/search/?clid=2186621&taxt=&lr=56&redircnt=1643944361.1'

    
    


if __name__ == "__main__":
    # Создание бота
    bot = telebot.TeleBot(token=API_TOKEN)
    # Для многопользовательской поддержки будет содержать идентификаторы чата, а также текущий язык вики
    current_chats = {}

    # Команда обработки /запуска
    @bot.message_handler(commands=['start'])
    def welcome(message):
        chat_id = message.chat.id  # Получение идентификатора чата

        wb = WikiBot()
        current_chats[chat_id] = wb
        current_chats[chat_id].wiki.set_lang("ru")  # установите язык по умолчанию

        # Приветственное сообщение
        bot.send_message(chat_id,
                         "Приятно познакомиться, {0.first_name}!\n"
                         "Меня зовут infobot, я бот, который позволяет тебе искать статьи в Википедии прямо в этом "
                         "чате.\n"
                         "Считай меня своей личной Википедией😉.\n"
                         "\nПиши любое слово, и давай начнем изучать!"""
                         "\n\n<b>Доступные команды:</b>"
                         "\n/start - запусти бота"
                         "\n/help - чтобы просмотреть доступные команды"
                         "\n/eng - поиск статей на английском языке"
                         "\n/rus - поиск статей на русском языке".format(message.from_user, bot.get_me()),
                         parse_mode='html')

    # Команда обработки /справки
    @bot.message_handler(commands=['help'])
    def command_help(message):
        chat_id = message.chat.id  # Получение идентификатора чата
        bot.send_message(chat_id, "\n\n<b>Доступные команды:</b>"
                                  "\n/start - запусти бота"
                                  "\n/help - чтобы просмотреть доступные команды"
                                  "\n/eng - поиск статей на английском языке"
                                  "\n/rus - поиск статей на русском языке",
                         parse_mode='html')

    # Обработка команды /ru
    @bot.message_handler(commands=['rus'])
    def change_lang_ru(message):
        chat_id = message.chat.id  # Получение идентификатора чата
        if chat_id not in current_chats.keys():
            bot.send_message(chat_id, "Пожалуйста, инициализируйте чат с /start")
        else:
            current_chats[chat_id].wiki.set_lang("ru")
            bot.send_message(chat_id, "Изменил язык википедии на русский")

    # Команда обработки /eng
    @bot.message_handler(commands=['eng'])
    def change_lang_ru(message):
        chat_id = message.chat.id  # Получение идентификатора чата
        if chat_id not in current_chats.keys():
            bot.send_message(chat_id, "Please use the /start command to begin the chat")
        else:
            current_chats[chat_id].wiki.set_lang("en")
            bot.send_message(chat_id, "Changed wikipedia language to English")

    # Обработка входящих сообщений
    @bot.message_handler(content_types=['text'])
    def get_text_messages(message):
        chat_id = message.chat.id  # Получение идентификатора чата
        if chat_id not in current_chats.keys():
            bot.send_message(chat_id, "Пожалуйста, используйте команду /start, чтобы начать чат")
        else:
            wik = getwiki(current_chats[chat_id].wiki, message.text)
            bot.send_message(chat_id, wik, parse_mode='html')


    # Поехали
    bot.polling(none_stop=True)